In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np


In [2]:
import pickle
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)


In [3]:
X

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [4]:
y


array([3, 8, 8, ..., 3, 3, 2])

# magic

In [16]:
from keras.layers import Input
from keras.models import Model

classes = 9

input_shape = (45,45,1)
i = Input(shape=input_shape)
x = Conv2D(32, (3,3), strides =2, activation = "relu")(i)
x = Conv2D(64, (3,3), strides = 2, activation = 'relu')(x)
x = Conv2D(128, (3,3), strides = 2, activation = 'relu')(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = Dense(classes , activation = 'softmax')(x)

model = Model(i,x)

# tragic

In [59]:

model = Sequential()

input_shape = (45,45,1)

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3)

In [17]:
#Compile and fit the model

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(X, y, 
                    epochs= 5, verbose = 1, callbacks=[callback])

Epoch 1/5
1699/1699 [==============================] - 27s 15ms/step - loss: 0.3608 - accuracy: 0.8965
Epoch 2/5
1699/1699 [==============================] - 25s 15ms/step - loss: 0.1836 - accuracy: 0.9439
Epoch 3/5
1699/1699 [==============================] - 25s 15ms/step - loss: 0.1448 - accuracy: 0.9549
Epoch 4/5
1699/1699 [==============================] - 26s 15ms/step - loss: 0.1193 - accuracy: 0.9628
Epoch 5/5
1699/1699 [==============================] - 25s 15ms/step - loss: 0.1010 - accuracy: 0.9681


In [24]:
pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)
X_test.shape

(118595, 45, 45)

In [19]:
%%time
predictions = model.predict(X_test)


CPU times: user 48.5 s, sys: 27.7 s, total: 1min 16s
Wall time: 12.9 s


In [20]:
predictions[0]

array([2.2587733e-08, 1.6798886e-07, 3.2554951e-04, 1.5129801e-06,
       8.2834376e-05, 4.9336329e-08, 9.6353018e-05, 5.1297884e-05,
       9.9944216e-01], dtype=float32)

In [21]:
#for uploading

import pandas as pd

pred = pd.DataFrame(predictions, columns = ["Type 0", "Type 1", "Type 2", "Type 3", "Type 4", "Type 5", "Type 6", "Type 7", "Type 8"])
pred


,Type 0,Type 1,Type 2,Type 3,Type 4,Type 5,Type 6,Type 7,Type 8
0,2.258773e-08,1.679889e-07,0.000326,1.512980e-06,0.000083,4.933633e-08,0.000096,5.129788e-05,0.999442
1,2.095160e-08,3.929562e-09,0.000138,4.141957e-06,0.000008,3.139956e-08,0.000237,2.287475e-07,0.999613
2,6.028895e-10,1.654213e-08,0.000071,8.023702e-08,0.000032,2.096528e-08,0.000260,2.734074e-05,0.999610
3,2.194049e-09,2.566946e-07,0.001401,1.192820e-05,0.008240,1.793361e-07,0.028118,3.580632e-06,0.962225
4,3.571650e-09,2.053476e-10,0.000168,2.036148e-06,0.000003,3.647719e-09,0.000043,3.443892e-07,0.999784
...,...,...,...,...,...,...,...,...,...
118590,8.762439e-06,5.903379e-07,0.760771,2.337284e-01,0.004688,1.639956e-04,0.000537,4.968376e-07,0.000102
118591,3.842103e-04,1.011243e-05,0.752240,2.053792e-01,0.002055,1.446058e-04,0.004391,1.557556e-05,0.035381
118592,1.656583e-08,1.968670e-09,0.134505,8.653919e-01,0.000017,2.252243e-07,0.000051,2.386395e-11,0.000034
118593,8.587016e-08,3.391424e-09,0.818932,1.805218e-01,0.000410,1.942884e-05,0.000044,9.502308e-09,0.000073


In [23]:
pred.to_csv('submission.csv', index_label = 'Id')